In [1]:
import pandas as pd

## Inspect available sheets

In [2]:
file_path = "online_retail_II.xlsx"
xls = pd.ExcelFile(file_path)

xls.sheet_names

['Year 2009-2010', 'Year 2010-2011']

## Load both years

In [3]:
df_2009 = pd.read_excel(file_path, sheet_name="Year 2009-2010")
df_2010 = pd.read_excel(file_path, sheet_name="Year 2010-2011")

## Combine into one dataset

In [4]:
df = pd.concat([df_2009, df_2010], ignore_index=True)

In [6]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [20]:
## Observation: The two sheets represent consecutive transactional periods and were merged to create a continuous observation window

## 2) CLEANING

In [124]:
# Ensure correct types
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Remove rows without CustomerID
df = df[df["Customer ID"].notna()].copy()

# Remove invalid sales: returns/cancellations/errors
df = df[(df["Quantity"] > 0) & (df["Price"] > 0)].copy()

# Make invoice unique across sheets (prevents duplicate invoice numbers)
df["Invoice"] = df["Invoice"].astype(str)
df["InvoiceID"] = df["Sheet"] + "_" + df["Invoice"]

# Revenue per line
df["TransactionValue"] = df["Quantity"] * df["Price"]

## 3) RFM TABLE

In [126]:
snapshot_date = df["InvoiceDate"].max() + pd.Timedelta(days=1)

rfm = (
    df.groupby("Customer ID")
      .agg(
          Recency=("InvoiceDate", lambda x: (snapshot_date - x.max()).days),
          Frequency=("InvoiceID", "nunique"),
          Monetary=("TransactionValue", "sum")
      )
      .reset_index()
)

# Quick sanity check
invalid_counts = (rfm[["Recency", "Frequency", "Monetary"]] <= 0).sum()
print("Invalid counts (should be 0):\n", invalid_counts)

Invalid counts (should be 0):
 Recency      0
Frequency    0
Monetary     0
dtype: int64


## 4) K-MEANS SEGMENTATION

In [133]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Log-transform skewed columns
rfm_seg = rfm.copy()
rfm_seg["Frequency_log"] = np.log1p(rfm_seg["Frequency"])
rfm_seg["Monetary_log"] = np.log1p(rfm_seg["Monetary"])

X = rfm_seg[["Recency", "Frequency_log", "Monetary_log"]].values
X_scaled = StandardScaler().fit_transform(X)

# Chosen k with a simple silhouette search
best_k = None
best_score = -1

for k in range(3, 7):  # keep it simple
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    if score > best_score:
        best_score = score
        best_k = k

print(f"Selected k = {best_k} (silhouette = {best_score:.3f})")

# Fit final model
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
rfm["Segment"] = kmeans.fit_predict(X_scaled)

# Segment profile (easy interpretation)
segment_profile = (
    rfm.groupby("Segment")
       .agg(
           Customers=("Customer ID", "count"),
           Recency_mean=("Recency", "mean"),
           Frequency_mean=("Frequency", "mean"),
           Monetary_mean=("Monetary", "mean"),
           Monetary_sum=("Monetary", "sum")
       )
       .sort_values("Monetary_sum", ascending=False)
       .reset_index()
)

print("\nSegment profile:\n", segment_profile)

Selected k = 3 (silhouette = 0.401)

Segment profile:
    Segment  Customers  Recency_mean  Frequency_mean  Monetary_mean  \
0        1       1706     58.915006       16.149472    8622.407498   
1        2       2337     91.585366        2.929825     853.417165   
2        0       1835    473.506812        1.823978     566.303037   

   Monetary_sum  
0  1.470983e+07  
1  1.994436e+06  
2  1.039166e+06  


In [131]:
# Rule-based labels based on relative ranking
profile = segment_profile.copy()
profile["Recency_rank"]   = profile["Recency_mean"].rank(method="dense", ascending=True).astype(int)
profile["Frequency_rank"] = profile["Frequency_mean"].rank(method="dense", ascending=False).astype(int)
profile["Monetary_rank"]  = profile["Monetary_mean"].rank(method="dense", ascending=False).astype(int)

worst_recency_rank = profile["Recency_rank"].max()
worst_freq_rank    = profile["Frequency_rank"].max()
worst_mon_rank     = profile["Monetary_rank"].max()

profile["Segment_Label"] = "Other"
profile.loc[
    (profile["Recency_rank"] == 1) & (profile["Frequency_rank"] == 1) & (profile["Monetary_rank"] == 1),
    "Segment_Label"
] = "Champions"
profile.loc[
    (profile["Recency_rank"] == worst_recency_rank) & ((profile["Monetary_rank"] <= 2) | (profile["Frequency_rank"] <= 2)),
    "Segment_Label"
] = "At Risk"
profile.loc[
    (profile["Frequency_rank"] == worst_freq_rank) & (profile["Monetary_rank"] == worst_mon_rank),
    "Segment_Label"
] = "Low Value"
profile.loc[
    (profile["Frequency_rank"] <= 2) & (profile["Monetary_rank"] <= 3) & (profile["Recency_rank"] > 1) & (profile["Segment_Label"] == "Other"),
    "Segment_Label"
] = "Loyal"

# Attached labels to customers
rfm = rfm.merge(profile[["Segment", "Segment_Label"]], on="Segment", how="left")

## 5) EXPORT FOR POWER BI

In [132]:
df.to_csv("fact_transactions.csv", index=False)
rfm.to_csv("rfm_segments.csv", index=False)
profile.to_csv("segment_profile.csv", index=False)

print("\nExported:")
print("- fact_transactions.csv")
print("- rfm_segments.csv")
print("- segment_profile.csv")


Exported:
- fact_transactions.csv
- rfm_segments.csv
- segment_profile.csv
